**HW 03**  
> In this homework, we will use MapReduce to calculate two canonical quantities in data analyses: the term frequency-inverse document frequency (tf-idf) measure, and the loss function for support vector machine.  
> In this homework, you should write your own functions to calculate the quantities, instead of using functions from pyspark's MLLib library. Your code should utilize the RDDs and dataframes created from pyspark.  
> While you probably will be able to use Chat to generate all of the necessary functions, I would encourage you to give it a try to design and process through how you may do it, before asking Chat.

In [1]:
# import necessary packages
from pyspark.sql import SparkSession, Row
import pyspark.sql.functions as F
from pyspark.sql.types import ArrayType, StringType

import numpy as np

***TASK 01***

In [2]:
# set up pyspark:
spark = (SparkSession.builder
         .master("local[*]")
         .appName("AG news")
         .getOrCreate()
        )

sc = spark.sparkContext

# ignore warnings:
spark.sparkContext.setLogLevel("ERROR")

# read in csv:
agnews = spark.read.csv("data/agnews_clean.csv", inferSchema=True, header=True)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/25 02:26:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# fix headers:
agnews = agnews.withColumnRenamed('_c0', 'id')
agnews = agnews.withColumn('filtered', F.from_json('filtered', ArrayType(StringType())))

In [4]:
# map function for term-frequency computation:
def tf_map(row):
    
    id = row['id']
    terms = [term.lower() for term in row['filtered']]

    length = len(terms)        # divide by length to get term frequency, not count!

    for term in terms:
        yield ((term, id), 1/length) 

In [5]:
# map/reduce term-frequency dataframe using spark
tf_data = (agnews.rdd
            .flatMap(tf_map)                      # tells spark we are applying over each row
            .reduceByKey(lambda a, b: a+b)        # reduces by like keys
            .collect()
            )

# returns ((term, doc), idf)

In [6]:
# map function for inverse document frequency computation:
def idf_map(row):
    
    id = row['id']
    terms = [term.lower() for term in row['filtered']]
    terms = set(terms)                            # want only unique terms

    for term in terms:
        yield (term, 1) 

# IDF = log(#docs / #docs_term)

# (term, count) ==> yield per doc, sum count of each doc
# --> sum count over each term

In [7]:
# count number of documents:
num_docs = agnews.count()

# map/reduce inverse document frequency dataframe using spark
idf_data = (agnews.rdd
            .flatMap(idf_map)                     # tells spark we are applying over each row
            .reduceByKey(lambda a, b: a+b)        # reduces by like keys (COUNT)
            .mapValues(lambda x: np.log(num_docs / x)) # use map again to divide D by x and take log
            .collect()
            )

# returns (term, idf)

In [8]:
# Calculate tf-idf measure for each row in the agnews_clean.csv. Save the measures in a new column.
    # tf_data --> ((term, doc), tf)
    # idf_data --> (term, idf)

# PARALLELIZE DATA: (too slow to compute linearly when attempted before)
tf_rdd = sc.parallelize(tf_data)  # ((term, doc), tf)
idf_rdd = sc.parallelize(idf_data)  # (term, idf)


## JOIN DATASETS:
# create new rdd for tf with key = term
tf_by_term = tf_rdd.map(lambda x: (x[0][0], (x[0][1], x[1])))  # returns: (term, (doc_id, tf))

# join with idf now that keys are same
tfidf_rdd = tf_by_term.join(idf_rdd)                           # returns: (term, ((doc_id, tf), idf))

# remap dataframe and calculate tf-idf
tfidf_rdd = tfidf_rdd.map(lambda x: ((x[0], x[1][0][0]), x[1][0][1] * x[1][1]))  # returns: ((term, doc), tf-idf)


## CREATE VECTORS OF TF-IDF:
# group by doc
tfidf_by_doc = tfidf_rdd.map(lambda x: (x[0][1], (x[0][0], x[1])))

# create vectors of tf_idf values per document
tfidf_vectors = tfidf_by_doc.groupByKey().mapValues(lambda keys: [tf_idf for term, tf_idf in keys]).sortBy(lambda x: x[0])

In [ ]:
# ADD VECTORS AS A NEW COLUMN 'TF-IDF' TO THE DATAFRAME:

# create clean dataframe with float values (error before):
tfidf_clean = tfidf_vectors.map(lambda x: (x[0], [float(score) for score in x[1]]))

# create dataframe of tf-idf values from vector and corresponding id:
tfidf_df = tfidf_clean.map(lambda x: Row(id=x[0], tf_idf=x[1])).toDF()

# join w/ old dataset (ON ID) into new dataset:
agnews_joined = agnews.join(tfidf_df, on='id', how='left')

In [12]:
# print out tf-idf values for first five documents:
agnews_joined.select('tf_idf').show(5, truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|tf_idf                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

***TASK 02***

In [13]:
# set up pyspark:
spark2 = (SparkSession.builder
         .master("local[*]")
         .appName("SVM obj")
         .getOrCreate()
        )

sc = spark2.sparkContext

# ignore warnings:
spark2.sparkContext.setLogLevel("ERROR")

# read in csv files:
w = spark2.read.csv('data/w.csv', header=False, inferSchema=True)
bias = spark2.read.csv('data/bias.csv', header=False, inferSchema=True)
data_svm = spark2.read.csv('data/data_for_svm.csv', header=False, inferSchema=True)

In [14]:
# 1. Design the MapReduce functions required to calculate the SVM loss function.

# map loss function (summation):
def map_loss(w, b, X_i, y_i):
    
    # calculate y_i(wT_x_i + b):
    prod = y_i * (np.dot(w, X_i) + b) 
    
    # choose max:
    loss = max(0, 1-prod)
    
    return loss

# reduce loss function (avg):
def reduce_loss(losses):
    return sum(losses)/len(losses)

In [15]:
# 2. Implement a function loss_SVM(w, b, X, y) that computes the SVM objective for given w and b using the dataset stored in X and y.

# w: weight vector
# b: bias 
# X: features matrix
# y: labels vector
# lamb: lambda value

def loss_SVM(w, b, X, y, lamb):

    # compute regularization: --> L2 norm = sqrt(dot prod)
    reg = lamb + np.dot(w, w) 
    
    # map loss:
    losses = [map_loss(X[i], y[i], w, b) for i in range(len(X))]

    # reduce loss: 
    loss = reduce_loss(losses)

    return loss + reg

In [16]:
# 3. Use the dataset data_for_svm.csv, where:
#    The first 64 columns contain the features (X)
#    The last column contains the labels (y)

# 4. Use the provided weights and bias (w.csv and bias.csv) to calculate the objective value:

# convert variables to usable forms (np.arrays):
w_array = np.array(w.rdd.first())
bias_array = bias.rdd.first()[0]

X_array = np.array([row[:-1] for row in data_svm.rdd.collect()])
y_array = np.array([row[-1] for row in data_svm.rdd.collect()])

In [18]:
# CALCULATE AND REPORT OBJECTIVE VALUE:
objective_value = loss_SVM(w_array, bias_array, X_array, y_array, 0.1)

print(f"Objective value is: {objective_value}")

Objective value is: 1.1032170403441435


In [19]:
# 5. Design the MapReduce function required to make predictions using the SVM model:

# prediction: if w * X_i + b >= 0 --> 1, else -1
def map_pred(w, b, X_i):
    
    # calculate w * X_i + b:
    score = np.dot(w, X_i) + b

    # return corresponding prediction:
    return 1 if score >= 0 else -1

# # don't need reduce function- nothing to 'reduce'
# def reduce_pred():
#     pass

In [20]:
# 6. Predict on all of the data using the provided weights and bias:

# create function to predict labels given w, b, X
def predict(w, b, X):
    return [map_pred(w, b, X_i) for X_i in X]

# calculate predictions with provided inputs
predictions = predict(w_array, bias_array, X_array)

# # print predictions:
# print(f"predictions: {predictions}")